In [1]:
from datetime import datetime, timedelta
from collections import defaultdict
import calendar


In [2]:
def get_billing_period(item_start, item_end, month_start, month_end):
    start = max(item_start, month_start)
    end = min(item_end, month_end)
    return (start, end) if start <= end else None


In [6]:
def generate_monthly_bill(item_list: list, target_month: str) -> dict:
    year, month = map(int, target_month.split("-"))
    month_start = datetime(year, month, 1)
    last_day = calendar.monthrange(year, month)[1]
    month_end = datetime(year, month, last_day)

    grouped_data = defaultdict(lambda: {"qty": 0, "amount": 0.0})

    for item in item_list:
        # Step 4.1: Parse dates
        start_date = datetime.strptime(item["start_date"], "%Y-%m-%d")
        stop_date = datetime.strptime(item["stop_date"], "%Y-%m-%d")

        # Step 4.2: Check if item is active during the target month
        billing_period = get_billing_period(start_date, stop_date, month_start, month_end)
        if not billing_period:
            continue  # Skip inactive items

        # Step 4.3: Calculate active days
        active_days = (billing_period[1] - billing_period[0]).days + 1
        days_in_month = (month_end - month_start).days + 1

        # Step 4.4: Normalize types
        qty = int(item["qty"])
        rate = float(item["rate"])

        # Step 4.5: Calculate prorated amount
        prorated_amount = (qty * rate) * (active_days / days_in_month)

        # Step 4.6: Create group key
        billing_period_str = billing_period[0].strftime("%Y-%m-%d") + " to " + billing_period[1].strftime("%Y-%m-%d")
        key = (item["item_code"], rate, billing_period_str)

        # Step 4.7: Group the items
        grouped_data[key]["qty"] += qty
        grouped_data[key]["amount"] += prorated_amount

    # Step 5: Prepare final output
    line_items = []
    total_revenue = 0.0

    for (item_code, rate, billing_period), values in grouped_data.items():
        amount = round(values["amount"], 2)
        line_items.append({
            "item_code": item_code,
            "rate": rate,
            "qty": values["qty"],
            "amount": amount,
            "billing_period": billing_period
        })
        total_revenue += amount

    return {
        "line_items": line_items,
        "total_revenue": round(total_revenue, 2)
    }


In [9]:
item_list = [ 
 
    { 
        "idx": 1, 
        "item_code": "Executive Desk (4*2)", 
        "sales_description": "Dedicated Executive Desk", 
        "qty": 10, 
        "rate": "1000", 
        "amount": "10000", 
        "start_date": "2023-11-01", 
        "stop_date": "2024-10-17", 
    }, 
    { 
        "idx": 2, 
        "item_code": "Executive Desk (4*2)", 
        "qty": "10", 
        "rate": "1080", 
        "amount": "10800", 
        "start_date": "2024-10-18", 
        "stop_date": "2025-10-31", 
    }, 
    { 
        "idx": 3, 
        "item_code": "Executive Desk (4*2)", 
        "qty": 15, 
        "rate": "1080", 
        "amount": "16200", 
        "start_date": "2024-11-01", 
        "stop_date": "2025-10-31", 
    }, 
    { 
        "idx": 4, 
        "item_code": "Executive Desk (4*2)", 
        "qty": 5, 
        "rate": "1000", 
        "amount": "5000", 
        "start_date": "2024-11-01", 
        "stop_date": "2025-10-31", 
    }, 
    { 
        "idx": 5, 
        "item_code": "Manager Cabin", 
        "qty": 5, 
        "rate": 5000, 
        "amount": 25000, 
        "start_date": "2024-11-01", 
        "stop_date": "2025-10-31", 
    }, 
    { 
        "idx": 6, 
        "item_code": "Manager Cabin", 
        "qty": 7, 
        "rate": "5000", 
        "amount": 35000, 
        "start_date": "2024-12-15", 
        "stop_date": "2025-10-31", 
    }, 
    { 
        "idx": 7, 
        "item_code": "Manager Cabin", 
        "qty": 10, 
        "rate": 4600, 
        "amount": 46000, 
        "start_date": "2023-11-01", 
        "stop_date": "2024-10-17", 
    }, 
    { 
        "idx": 8, 
        "item_code": "Parking (2S)", 
        "qty": 10, 
        "rate": 1000, 
        "amount": 10000, 
        "start_date": "2024-11-01", 
        "stop_date": "2025-10-31", 
    }, 
    { 
        "idx": 9, 
        "item_code": "Parking (2S)", 
        "qty": 10, 
        "rate": 0, 
        "amount": 0, 
        "start_date": "2024-11-01", 
        "stop_date": "2025-10-31", 
    }, 
    { 
        "idx": 10, 
        "item_code": "Executive Desk (4*2)", 
        "qty": "8", 
        "rate": "1100", 
        "amount": "8800", 
        "start_date": "2024-11-15", 
        "stop_date": "2025-01-31", 
    }, 
    { 
        "idx": 11, 
        "item_code": "Manager Cabin", 
        "qty": "3", 
        "rate": "5200", 
        "amount": "15600", 
        "start_date": "2024-10-10", 
        "stop_date": "2024-11-10", 
    }, 
    { 
        "idx": 12, 
        "item_code": "Conference Table", 
        "qty": 1, 
        "rate": "20000", 
        "amount": "20000", 
        "start_date": "2024-11-05", 
        "stop_date": "2024-11-20", 
    }, 
    { 
        "idx": 13, 
        "item_code": "Parking (2S)", 
        "qty": 5, 
        "rate": "1000", 
        "amount": "5000", 
        "start_date": "2024-11-15", 
        "stop_date": "2025-02-28", 
    }, 
    { 
        "idx": 14, 
        "item_code": "Reception Desk", 
        "qty": 2, 
        "rate": "7000", 
        "amount": "14000", 
        "start_date": "2024-11-01", 
        "stop_date": "2025-03-31", 
    }, 
    { 
        "idx": 15, 
        "item_code": "Reception Desk", 
        "qty": 1, 
        "rate": "7000", 
        "amount": "7000", 
        "start_date": "2024-11-10", 
        "stop_date": "2024-11-25", 
    }, 
    { 
        "idx": 16, 
        "item_code": "Breakout Area", 
        "qty": 3, 
        "rate": "3000", 
        "amount": "9000", 
        "start_date": "2024-01-01", 
        "stop_date": "2024-01-31", 
    } 
 
]


In [10]:
result = generate_monthly_bill(item_list, "2024-11")
result


{'line_items': [{'item_code': 'Executive Desk (4*2)',
   'rate': 1080.0,
   'qty': 25,
   'amount': 27000.0,
   'billing_period': '2024-11-01 to 2024-11-30'},
  {'item_code': 'Executive Desk (4*2)',
   'rate': 1000.0,
   'qty': 5,
   'amount': 5000.0,
   'billing_period': '2024-11-01 to 2024-11-30'},
  {'item_code': 'Manager Cabin',
   'rate': 5000.0,
   'qty': 5,
   'amount': 25000.0,
   'billing_period': '2024-11-01 to 2024-11-30'},
  {'item_code': 'Parking (2S)',
   'rate': 1000.0,
   'qty': 10,
   'amount': 10000.0,
   'billing_period': '2024-11-01 to 2024-11-30'},
  {'item_code': 'Parking (2S)',
   'rate': 0.0,
   'qty': 10,
   'amount': 0.0,
   'billing_period': '2024-11-01 to 2024-11-30'},
  {'item_code': 'Executive Desk (4*2)',
   'rate': 1100.0,
   'qty': 8,
   'amount': 4693.33,
   'billing_period': '2024-11-15 to 2024-11-30'},
  {'item_code': 'Manager Cabin',
   'rate': 5200.0,
   'qty': 3,
   'amount': 5200.0,
   'billing_period': '2024-11-01 to 2024-11-10'},
  {'item_code'